# Обнавлённый обработчик
#### 1. Базовые признаки пакета:
 1. Временная метка (timestamp)
 2. MAC-адрес отправителя (source MAC)
 3. MAC-адрес получателя (destination MAC)
 4. IP-адрес отправителя (source IP)
 5. IP-адрес получателя (destination IP)
 6. Тип ARP-операции (request/reply)
#### 2. Признаки на основе дубликатов (критичные для вашего случая):
 1. Количество дубликатов в заданном временном окне (например, за 10 секунд)
 2. Процент дублирующихся пакетов
 3. Интервал между дубликатами
 4. Количество ответов без запросов
#### 3. Признаки на основе соответствия MAC-IP:
 1. Изменение связки MAC-IP для одного IP-адреса
 2. Количество разных MAC-адресов, связанных с одним IP
 3. Частота изменения MAC для IP-адреса
#### 4. Временные признаки:
 1. Скорость поступления ARP-пакетов (пакетов в секунду)
 2. Интервал между запросами и ответами
 3. Время суток (может быть полезно для обнаружения аномальной активности в нехарактерное время)
#### 5. Признаки на основе направленности:
 1. Количество ARP-запросов к шлюзу
 2. Количество broadcast-запросов от одного источника
 3. Соотношение unicast/broadcast ARP-сообщений
#### 6. Контекстные признаки:
 1. Наличие других типов трафика от тех же MAC-адресов
 2. Соответствие обнаруженного поведения типичным паттернам сети
 3. Эти признаки основаны на:
 4. Анализе ваших успешных атак (особенно с высоким процентом дубликатов)
 5. Различных типах собранных данных (Ettercap, Bettercap)
 6. Характерных признаках ARP-spoofing атак

In [1]:
from scapy.all import *
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os
import warnings
warnings.filterwarnings('ignore')

class ARPFeatureExtractor:
    def __init__(self, window_size=10):
        """
        Инициализация экстрактора признаков
        window_size: размер временного окна в секундах для расчета признаков
        """
        self.window_size = window_size
        self.features = []
        
    def extract_basic_features(self, packet):
        """Извлечение базовых признаков из пакета"""
        if ARP in packet:
            return {
                'timestamp': packet.time,
                'src_mac': packet[Ether].src,
                'dst_mac': packet[Ether].dst,
                'src_ip': packet[ARP].psrc,
                'dst_ip': packet[ARP].pdst,
                'opcode': packet[ARP].op,  # 1 для запроса, 2 для ответа
                'is_broadcast': packet[Ether].dst == 'ff:ff:ff:ff:ff:ff'
            }
        return None

    def calculate_window_features(self, packets):
        """Расчет признаков в скользящем окне"""
        window_features = []
        
        for i in range(len(packets)):
            current_time = packets[i].time
            window_packets = [p for p in packets 
                            if current_time - self.window_size <= p.time <= current_time]
            
            if not window_packets:
                continue
                
            # Подсчет дубликатов
            packet_hashes = [hash(f"{p[ARP].psrc}-{p[ARP].pdst}-{p[ARP].op}") 
                           for p in window_packets]
            duplicates = len(packet_hashes) - len(set(packet_hashes))
            
            # Подсчет запросов и ответов
            requests = sum(1 for p in window_packets if p[ARP].op == 1)
            replies = sum(1 for p in window_packets if p[ARP].op == 2)
            
            # Расчет частоты пакетов
            time_span = window_packets[-1].time - window_packets[0].time
            packet_rate = len(window_packets) / time_span if time_span > 0 else 0
            
            # MAC-IP соответствия
            mac_ip_pairs = {}
            for p in window_packets:
                if p[ARP].op == 2:  # только ответы
                    ip = p[ARP].psrc
                    mac = p[ARP].hwsrc
                    if ip in mac_ip_pairs:
                        mac_ip_pairs[ip].add(mac)
                    else:
                        mac_ip_pairs[ip] = {mac}
            
            # Количество IP с множественными MAC
            multiple_macs = sum(1 for macs in mac_ip_pairs.values() if len(macs) > 1)
            
            window_features.append({
                'duplicates': duplicates,
                'requests': requests,
                'replies': replies,
                'packet_rate': packet_rate,
                'multiple_macs': multiple_macs,
                'request_reply_ratio': requests / replies if replies > 0 else float('inf')
            })
            
        return window_features

    def process_pcap(self, pcap_file):
        """Обработка PCAP файла и извлечение признаков"""
        print(f"Обработка файла: {pcap_file}")
        packets = rdpcap(pcap_file)
        arp_packets = [p for p in packets if ARP in p]
        
        if not arp_packets:
            print(f"Предупреждение: В файле {pcap_file} нет ARP пакетов")
            return pd.DataFrame()
        
        # Извлечение базовых признаков
        basic_features = [self.extract_basic_features(p) for p in arp_packets]
        basic_features = [f for f in basic_features if f is not None]
        
        # Расчет признаков в окне
        window_features = self.calculate_window_features(arp_packets)
        
        # Объединение признаков
        for i in range(len(basic_features)):
            if i < len(window_features):
                basic_features[i].update(window_features[i])
        
        return pd.DataFrame(basic_features)

    def process_multiple_pcaps(self, pcap_files, label):
        """Обработка нескольких PCAP файлов"""
        all_features = []
        
        for pcap_file in pcap_files:
            try:
                df = self.process_pcap(pcap_file)
                if not df.empty:
                    df['label'] = label
                    df['session_id'] = os.path.basename(pcap_file)
                    all_features.append(df)
            except Exception as e:
                print(f"Ошибка при обработке {pcap_file}: {str(e)}")
        
        return pd.concat(all_features, ignore_index=True) if all_features else pd.DataFrame()

    def save_to_csv(self, df, output_file):
        """Сохранение признаков в CSV файл"""
        if not df.empty:
            df.to_csv(output_file, index=False)
            print(f"Признаки сохранены в {output_file}")
        else:
            print(f"Предупреждение: Нет данных для сохранения в {output_file}")

def main():
    # Создание директорий для результатов
    os.makedirs("results", exist_ok=True)
    
    # Пути к файлам
    normal_files = [
        r"D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_2230.pcapng",
        r"D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_1190.pcapng",
        r"D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_1045.pcapng",
        r"D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_910.pcapng",
        r"D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_388.pcapng"
    ]
    
    anomaly_files = [
        r"D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\A_652.pcapng",
        r"D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\A_882.pcapng",
        r"D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\A_4301.pcapng"
    ]
    
    # Создание экстрактора
    extractor = ARPFeatureExtractor(window_size=10)
    
    # Обработка нормального трафика
    print("\n=== Обработка нормального трафика ===")
    normal_df = extractor.process_multiple_pcaps(normal_files, label=0)
    extractor.save_to_csv(normal_df, "results/normal_features.csv")
    
    # Обработка аномального трафика
    print("\n=== Обработка аномального трафика ===")
    anomaly_df = extractor.process_multiple_pcaps(anomaly_files, label=1)
    extractor.save_to_csv(anomaly_df, "results/anomaly_features.csv")
    
    # Объединение датасетов
    print("\n=== Объединение датасетов ===")
    combined_df = pd.concat([normal_df, anomaly_df], ignore_index=True)
    extractor.save_to_csv(combined_df, "results/combined_features.csv")
    
    # Вывод статистики
    print("\n=== Статистика обработки ===")
    print(f"Всего пакетов в нормальном трафике: {len(normal_df)}")
    print(f"Всего пакетов в аномальном трафике: {len(anomaly_df)}")
    print(f"Всего пакетов в объединенном датасете: {len(combined_df)}")
    
    # Статистика по сессиям
    print("\n=== Статистика по сессиям ===")
    print("\nНормальный трафик:")
    print(normal_df.groupby('session_id').size())
    print("\nАномальный трафик:")
    print(anomaly_df.groupby('session_id').size())

if __name__ == "__main__":
    main()

c:\Users\Slyexistence\AppData\Local\Programs\Python\Python312\Lib\site-packages\scapy\layers\ipsec.py:512: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  cipher=algorithms.TripleDES,
c:\Users\Slyexistence\AppData\Local\Programs\Python\Python312\Lib\site-packages\scapy\layers\ipsec.py:516: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  cipher=algorithms.TripleDES,



=== Обработка нормального трафика ===
Обработка файла: D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_2230.pcapng
Обработка файла: D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_1190.pcapng
Обработка файла: D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_1045.pcapng
Обработка файла: D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_910.pcapng
Обработка файла: D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\N_388.pcapng
Признаки сохранены в results/normal_features.csv

=== Обработка аномального трафика ===
Обработка файла: D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\A_652.pcapng
Обработка файла: D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\A_882.pcapng
Обработка файла: D:\Проекты\Дипломаня работа\DoFitN\Data\only_arp\A_4301.pcapng
Признаки сохранены в results/anomaly_features.csv

=== Объединение датасетов ===
Признаки сохранены в results/combined_features.csv

=== Статистика обработки ===
Всего пакетов в нормальном трафике: 5763
Всего пакетов в аномальном трафике

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Dense, LSTM, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import warnings
warnings.filterwarnings('ignore')

# Параметры
results_dir = "D:/Проекты/Дипломаня работа/DoFitN/Code/DoFitN/new_code/Test/AE2/results"
models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# 1. Загрузка данных
def load_and_preprocess_data():
    # Загрузка объединенного датасета
    df = pd.read_csv(f"{results_dir}/combined_features.csv")
    
    # Просмотр данных
    print("Распределение классов:")
    print(df['label'].value_counts())
    print("\nПервые несколько строк:")
    print(df.head())
    
    # Удаление ненужных столбцов
    features_to_drop = ['timestamp', 'session_id']
    X = df.drop(['label'] + features_to_drop, axis=1)
    y = df['label']
    
    # Кодирование категориальных признаков
    categorical_features = ['src_mac', 'dst_mac', 'src_ip', 'dst_ip']
    numerical_features = [col for col in X.columns if col not in categorical_features]
    
    # Числовые признаки
    X_numerical = X[numerical_features].copy()
    
    # Категориальные признаки - используем хеширование MAC и IP для уменьшения размерности
    for cat_feature in categorical_features:
        X_numerical[f'{cat_feature}_hash'] = X[cat_feature].apply(lambda x: hash(str(x)) % 100)
    
    # Удаляем оригинальные категориальные столбцы
    X = X_numerical
    
    # Замена inf и NaN значений
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(0)
    
    # Нормализация данных
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Разделение на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Подготовка данных для LSTM (reshape в 3D)
    # Для LSTM нам нужен формат [samples, time_steps, features]
    # Мы будем использовать 1 временной шаг для простоты
    X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test_lstm = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    
    return X_train, X_test, X_train_lstm, X_test_lstm, y_train, y_test, scaler

# 2. Создание автоэнкодера
def create_autoencoder(input_dim):
    # Определение размеров слоев
    encoding_dim = 10
    
    # Входной слой
    input_layer = Input(shape=(input_dim,))
    
    # Энкодер
    encoder = Dense(32, activation='relu')(input_layer)
    encoder = Dense(16, activation='relu')(encoder)
    encoder = Dense(encoding_dim, activation='relu')(encoder)
    
    # Декодер
    decoder = Dense(16, activation='relu')(encoder)
    decoder = Dense(32, activation='relu')(decoder)
    decoder = Dense(input_dim, activation='sigmoid')(decoder)
    
    # Автоэнкодер (полная модель)
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    encoder_model = Model(inputs=input_layer, outputs=encoder)
    
    # Компиляция
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    
    return autoencoder, encoder_model

# 3. Создание LSTM модели
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# 4. Создание гибридной модели - автоэнкодер + LSTM
def create_hybrid_model(input_dim):
    # LSTM автоэнкодер
    # Входной слой
    timesteps = 1
    input_layer = Input(shape=(timesteps, input_dim))
    
    # Энкодер LSTM
    encoded = LSTM(32, return_sequences=True)(input_layer)
    encoded = LSTM(16, return_sequences=False)(encoded)
    
    # Декодер LSTM
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(16, return_sequences=True)(decoded)
    decoded = LSTM(32, return_sequences=True)(decoded)
    decoded = TimeDistributed(Dense(input_dim))(decoded)
    
    # Автоэнкодер (полная модель)
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')
    
    # Классификатор
    classifier_input = Input(shape=(timesteps, input_dim))
    classifier_encoded = LSTM(32, return_sequences=True)(classifier_input)
    classifier_encoded = LSTM(16)(classifier_encoded)
    classifier_dense = Dense(8, activation='relu')(classifier_encoded)
    classifier_output = Dense(1, activation='sigmoid')(classifier_dense)
    
    classifier = Model(inputs=classifier_input, outputs=classifier_output)
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return autoencoder, classifier

# 5. Функция для обучения моделей
def train_models(X_train, X_test, X_train_lstm, X_test_lstm, y_train, y_test):
    input_dim = X_train.shape[1]
    lstm_input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])
    
    # Колбэки
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # 5.1 Обучение автоэнкодера
    print("Обучение автоэнкодера...")
    autoencoder, encoder = create_autoencoder(input_dim)
    history_ae = autoencoder.fit(
        X_train, X_train,
        epochs=50,
        batch_size=64,
        validation_data=(X_test, X_test),
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Получение признаков от автоэнкодера
    encoded_train = encoder.predict(X_train)
    encoded_test = encoder.predict(X_test)
    
    # Расчет ошибки реконструкции
    train_predictions = autoencoder.predict(X_train)
    test_predictions = autoencoder.predict(X_test)
    
    train_mse = np.mean(np.power(X_train - train_predictions, 2), axis=1)
    test_mse = np.mean(np.power(X_test - test_predictions, 2), axis=1)
    
    # Определение порога для автоэнкодера
    threshold = np.percentile(train_mse, 95)  # Используем 95-й перцентиль как порог
    
    # Предсказания автоэнкодера
    y_pred_ae = (test_mse > threshold).astype(int)
    
    # 5.2 Обучение LSTM
    print("\nОбучение LSTM...")
    lstm_model = create_lstm_model(lstm_input_shape)
    history_lstm = lstm_model.fit(
        X_train_lstm, y_train,
        epochs=50,
        batch_size=64,
        validation_data=(X_test_lstm, y_test),
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Предсказания LSTM
    y_pred_proba_lstm = lstm_model.predict(X_test_lstm)
    y_pred_lstm = (y_pred_proba_lstm > 0.5).astype(int).flatten()
    
    # 5.3 Обучение гибридной модели
    print("\nОбучение гибридной модели...")
    hybrid_ae, hybrid_classifier = create_hybrid_model(input_dim)
    
    # Обучение гибридного автоэнкодера
    history_hybrid_ae = hybrid_ae.fit(
        X_train_lstm, X_train_lstm,
        epochs=50,
        batch_size=64,
        validation_data=(X_test_lstm, X_test_lstm),
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Обучение гибридного классификатора
    history_hybrid_clf = hybrid_classifier.fit(
        X_train_lstm, y_train,
        epochs=50,
        batch_size=64,
        validation_data=(X_test_lstm, y_test),
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Предсказания гибридной модели
    y_pred_proba_hybrid = hybrid_classifier.predict(X_test_lstm)
    y_pred_hybrid = (y_pred_proba_hybrid > 0.5).astype(int).flatten()
    
    # Сохранение моделей
    autoencoder.save(f"{models_dir}/autoencoder.h5")
    lstm_model.save(f"{models_dir}/lstm_model.h5")
    hybrid_ae.save(f"{models_dir}/hybrid_ae.h5")
    hybrid_classifier.save(f"{models_dir}/hybrid_classifier.h5")
    
    return {
        'autoencoder': {
            'model': autoencoder,
            'encoder': encoder,
            'history': history_ae,
            'predictions': y_pred_ae,
            'threshold': threshold,
            'mse': test_mse
        },
        'lstm': {
            'model': lstm_model,
            'history': history_lstm,
            'predictions': y_pred_lstm,
            'probabilities': y_pred_proba_lstm
        },
        'hybrid': {
            'ae_model': hybrid_ae,
            'classifier_model': hybrid_classifier,
            'ae_history': history_hybrid_ae,
            'clf_history': history_hybrid_clf,
            'predictions': y_pred_hybrid,
            'probabilities': y_pred_proba_hybrid
        }
    }

# 6. Визуализация результатов
def visualize_results(models_data, y_test):
    # Создание фигуры для всех графиков
    plt.figure(figsize=(18, 12))
    
    # 6.1 Матрицы ошибок
    plt.subplot(2, 3, 1)
    cm_ae = confusion_matrix(y_test, models_data['autoencoder']['predictions'])
    sns.heatmap(cm_ae, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('Матрица ошибок: Автоэнкодер')
    plt.ylabel('Истинный класс')
    plt.xlabel('Предсказанный класс')
    
    plt.subplot(2, 3, 2)
    cm_lstm = confusion_matrix(y_test, models_data['lstm']['predictions'])
    sns.heatmap(cm_lstm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('Матрица ошибок: LSTM')
    plt.ylabel('Истинный класс')
    plt.xlabel('Предсказанный класс')
    
    plt.subplot(2, 3, 3)
    cm_hybrid = confusion_matrix(y_test, models_data['hybrid']['predictions'])
    sns.heatmap(cm_hybrid, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title('Матрица ошибок: Гибридная модель')
    plt.ylabel('Истинный класс')
    plt.xlabel('Предсказанный класс')
    
    # 6.2 График потерь при обучении
    plt.subplot(2, 3, 4)
    plt.plot(models_data['autoencoder']['history'].history['loss'], label='Обучение')
    plt.plot(models_data['autoencoder']['history'].history['val_loss'], label='Валидация')
    plt.title('Потери Автоэнкодера')
    plt.ylabel('Потери')
    plt.xlabel('Эпоха')
    plt.legend()
    
    plt.subplot(2, 3, 5)
    plt.plot(models_data['lstm']['history'].history['loss'], label='Обучение')
    plt.plot(models_data['lstm']['history'].history['val_loss'], label='Валидация')
    plt.title('Потери LSTM')
    plt.ylabel('Потери')
    plt.xlabel('Эпоха')
    plt.legend()
    
    plt.subplot(2, 3, 6)
    plt.plot(models_data['hybrid']['clf_history'].history['loss'], label='Обучение')
    plt.plot(models_data['hybrid']['clf_history'].history['val_loss'], label='Валидация')
    plt.title('Потери гибридной модели')
    plt.ylabel('Потери')
    plt.xlabel('Эпоха')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f"{models_dir}/model_performance.png", dpi=300)
    plt.close()
    
    # 6.3 График точности
    plt.figure(figsize=(10, 6))
    
    # Расчет точности
    acc_ae = accuracy_score(y_test, models_data['autoencoder']['predictions'])
    acc_lstm = accuracy_score(y_test, models_data['lstm']['predictions'])
    acc_hybrid = accuracy_score(y_test, models_data['hybrid']['predictions'])
    
    models = ['Автоэнкодер', 'LSTM', 'Гибридная']
    accuracies = [acc_ae, acc_lstm, acc_hybrid]
    
    plt.bar(models, accuracies, color=['blue', 'green', 'purple'])
    plt.ylim(0, 1.0)
    plt.title('Сравнение точности моделей')
    plt.ylabel('Точность')
    
    # Добавление значений на столбцы
    for i, v in enumerate(accuracies):
        plt.text(i, v + 0.02, f'{v:.2f}', ha='center')
    
    plt.tight_layout()
    plt.savefig(f"{models_dir}/accuracy_comparison.png", dpi=300)
    plt.close()
    
    # 6.4 Вывод отчета о классификации
    print("\nОтчет о классификации - Автоэнкодер:")
    print(classification_report(y_test, models_data['autoencoder']['predictions']))
    
    print("\nОтчет о классификации - LSTM:")
    print(classification_report(y_test, models_data['lstm']['predictions']))
    
    print("\nОтчет о классификации - Гибридная модель:")
    print(classification_report(y_test, models_data['hybrid']['predictions']))
    
    # 6.5 Распределение ошибок реконструкции
    plt.figure(figsize=(10, 6))
    normal_error = models_data['autoencoder']['mse'][y_test == 0]
    anomaly_error = models_data['autoencoder']['mse'][y_test == 1]
    
    plt.hist(normal_error, bins=50, alpha=0.5, label='Нормальный трафик')
    plt.hist(anomaly_error, bins=50, alpha=0.5, label='Аномальный трафик')
    plt.axvline(models_data['autoencoder']['threshold'], color='r', linestyle='--', 
                label=f'Порог ({models_data["autoencoder"]["threshold"]:.4f})')
    
    plt.title('Распределение ошибок реконструкции автоэнкодера')
    plt.xlabel('Средняя квадратичная ошибка')
    plt.ylabel('Частота')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{models_dir}/reconstruction_error.png", dpi=300)
    plt.close()

# Основная функция
def main():
    print("Загрузка и предобработка данных...")
    X_train, X_test, X_train_lstm, X_test_lstm, y_train, y_test, scaler = load_and_preprocess_data()
    
    print("\nОбучение моделей...")
    models_data = train_models(X_train, X_test, X_train_lstm, X_test_lstm, y_train, y_test)
    
    print("\nВизуализация результатов...")
    visualize_results(models_data, y_test)
    
    print("\nГотово! Результаты и модели сохранены в директории:", models_dir)

if __name__ == "__main__":
    main()

Загрузка и предобработка данных...
Распределение классов:
label
1    5835
0    5763
Name: count, dtype: int64

Первые несколько строк:
      timestamp            src_mac            dst_mac       src_ip  \
0  1.737294e+09  00:ad:24:bf:9d:52  ff:ff:ff:ff:ff:ff  192.168.1.1   
1  1.737294e+09  00:ad:24:bf:9d:52  ff:ff:ff:ff:ff:ff  192.168.1.1   
2  1.737294e+09  00:ad:24:bf:9d:52  ff:ff:ff:ff:ff:ff  192.168.1.1   
3  1.737294e+09  00:ad:24:bf:9d:52  ff:ff:ff:ff:ff:ff  192.168.1.1   
4  1.737294e+09  00:ad:24:bf:9d:52  ff:ff:ff:ff:ff:ff  192.168.1.1   

         dst_ip  opcode  is_broadcast  duplicates  requests  replies  \
0  192.168.1.72       1          True           0         1        0   
1  192.168.1.72       1          True           0         1        0   
2  192.168.1.72       1          True           0         1        0   
3  192.168.1.15       1          True           0         2        0   
4  192.168.1.15       1          True           1         3        0   

   packet_r


Визуализация результатов...

Отчет о классификации - Автоэнкодер:
              precision    recall  f1-score   support

           0       0.50      0.95      0.65      1153
           1       0.49      0.05      0.09      1167

    accuracy                           0.50      2320
   macro avg       0.49      0.50      0.37      2320
weighted avg       0.49      0.50      0.37      2320


Отчет о классификации - LSTM:
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      1153
           1       0.99      0.92      0.96      1167

    accuracy                           0.96      2320
   macro avg       0.96      0.96      0.96      2320
weighted avg       0.96      0.96      0.96      2320


Отчет о классификации - Гибридная модель:
              precision    recall  f1-score   support

           0       0.80      0.96      0.88      1153
           1       0.95      0.77      0.85      1167

    accuracy                           0.